Importing libraries

In [2]:
import numpy as np
import random

print(np.__version__)

2.2.6


Initializing bounds that were in the document

In [3]:
# Define the bounds as per the project
bounds = [
    (10, 30),    # x0: number of filters in convolutional layer 1
    (2, 3),      # x1: kernel size in convolutional layer 1
    (10, 20),    # x2: number of filters in convolutional layer 2
    (2, 3),      # x3: kernel size in convolutional layer 2
    (0.1, 0.3),  # x4: dropout_rate
    (32, 64),    # x5: number of neurons in dense layer 1
    (16, 32),    # x6: number of neurons in dense layer 2
    (-5, -3)     # x7: log10_learning_rate
]

### Initialization of Population
**initialize_population()**

This function creates the initial population for the genetic algorithm.
For each individual, it randomly generates gene values within the specified bounds and adds them to the population.
The output is a list of individuals that serve as the starting generation for the algorithm.


In [4]:
def initialize_population(pop_size, bounds):
    # Empty array
    population = []
    for _ in range(pop_size):
        # generating random values for individuals within the given bounds
        individual = [random.uniform(lower, upper) for lower, upper in bounds]
        # appending each individual to the population
        population.append(individual)
    return population

### Evaluation of Population

**evaluate_population()**

This function evaluates each individual in the population by applying the provided objective function and calculating its fitness value.
It stores each individual together with its fitness as a tuple, then sorts the list in ascending order to support minimization tasks.
The output is a sorted list of individuals paired with their fitness scores.


In [5]:
def evaluate_population(population, objective_function):
    evaluated = []
    for ind in population:
        fitness = objective_function(ind)
        evaluated.append((ind, fitness))
    evaluated.sort(key=lambda x: x[1])  # Sort by fitness ascending (minimize)
    return evaluated

### Parent Selection

**select_parents()**

This function selects two parents using **Tournament Selection**.
It randomly samples a small group of individuals, compares their fitness, and chooses the best one as a parent.
This process is repeated to select the second parent.

**Tournament Selection** is fast and easy to implement. It maintains good selection pressure by favoring fitter individuals while still giving weaker ones a small chance.

Compared to other methods:

* **Roulette Wheel** depends on proportional fitness and may be biased if values are close.
* **Rank Selection** reduces dominance of top individuals but can slow convergence.
  Tournament Selection balances simplicity, speed, and robustness.


In [6]:
def select_parents(evaluated_population, tournament_size=3):
    def tournament():
        candidates = random.sample(evaluated_population, tournament_size)
        candidates.sort(key=lambda x: x[1])  # Sort by fitness
        return candidates[0][0]  # Best in tournament
    parent1 = tournament()
    parent2 = tournament()
    return parent1, parent2

### Crossover Operation

**crossover()**

This function performs crossover between two parents to produce two new offspring.
With a given probability, **Uniform Crossover** is used, where each gene has a 50% chance of being taken from either parent, creating high genetic mixing.
If the crossover rate condition is not met, the parents are copied directly without modification.

**Uniform Crossover** provides greater variability because every gene is independently chosen.
Compared to other methods:

* **Single-Point** crossover is simpler but swaps large gene blocks, reducing diversity.
* **Two-Point** crossover offers more mixing than single-point but still limited to segment swaps.
  Uniform crossover maximizes exploration by distributing genes more randomly.


In [7]:
def crossover(parent1, parent2, crossover_rate=0.9):
    if random.random() > crossover_rate:
        return parent1[:], parent2[:]  # No crossover
    child1 = []
    child2 = []
    for i in range(len(parent1)):
        if random.random() < 0.5:
            child1.append(parent1[i])
            child2.append(parent2[i])
        else:
            child1.append(parent2[i])
            child2.append(parent1[i])
    return child1, child2

### Mutation Operation

**mutate()**

This function introduces small random changes to an individual to maintain diversity and explore new solutions.
Using **Gaussian Mutation**, each gene has a probability (`mutation_rate`) of being altered by adding noise drawn from a normal distribution scaled by `mutation_strength`.

* **mutation_rate** — controls **how often** genes are mutated.
* **mutation_strength** — controls **how strong the change** is (size of added noise).

Gaussian mutation allows smooth, controlled adjustments, making it effective for continuous optimization.
Compared to other methods:

* **Random Resetting** replaces a value completely, causing large jumps.
* **Uniform Mutation** adds fixed-range noise, lacking fine-grained control.
  Gaussian mutation provides more natural and gradual evolution.


In [8]:
def mutate(individual, mutation_rate=0.1, bounds=bounds, mutation_strength=0.1):
    mutated = individual[:]
    for i in range(len(mutated)):
        if random.random() < mutation_rate:
            lower, upper = bounds[i]
            sigma = (upper - lower) * mutation_strength
            mutated[i] += random.gauss(0, sigma)
            mutated[i] = max(lower, min(upper, mutated[i]))  # Clip to bounds
    return mutated

### Full Genetic Algorithm with Elitism

**genetic_algorithm()**

This function implements an **Elitist Genetic Algorithm**, which evolves a population over multiple generations to minimize the objective function.
At each generation, the best-performing individuals (elites) are copied directly into the next generation, ensuring strong solutions are preserved.

The algorithm performs the following steps:

* **Initialization**: Generate the first population randomly.
* **Evaluation**: Compute fitness for all individuals.
* **Elitism**: Keep the top `elite_size` individuals unchanged.
* **Parent Selection**: Choose parents using the selection method.
* **Crossover & Mutation**: Create new offspring and introduce variation.
* **Tracking Progress**: Record the best fitness value every generation for visualization.

Elitism provides faster convergence by safeguarding high-quality solutions, while the remaining genetic operators continue exploring new areas of the search space.


In [9]:
def genetic_algorithm(objective_function, pop_size=20, elite_size=2, generations=15, crossover_rate=0.9, mutation_rate=0.1, mutation_strength=0.1):
    # Initialize population
    population = initialize_population(pop_size, bounds)
    evaluated_pop = evaluate_population(population, objective_function)
    best_individual, best_fitness = evaluated_pop[0]
    
    # Track best fitness history (starting with initial)
    best_fitness_history = [best_fitness]
    
    for gen in range(generations):
        print(f"Generation {gen+1}")
        # Elites
        elites = [ind for ind, fit in evaluated_pop[:elite_size]]
        # Generate new population
        new_population = elites[:]
        while len(new_population) < pop_size:
            parent1, parent2 = select_parents(evaluated_pop)
            child1, child2 = crossover(parent1, parent2, crossover_rate)
            child1 = mutate(child1, mutation_rate, bounds, mutation_strength)
            child2 = mutate(child2, mutation_rate, bounds, mutation_strength)
            new_population.append(child1)
            if len(new_population) < pop_size:
                new_population.append(child2)
        
        # Evaluate new population
        evaluated_pop = evaluate_population(new_population, objective_function)
        current_best, current_fitness = evaluated_pop[0]
        if current_fitness < best_fitness:
            best_individual, best_fitness = current_best, current_fitness
            print(f"New best fitness: {best_fitness}")
        
        # Append the best fitness so far to history
        best_fitness_history.append(best_fitness)
    
    return best_individual, best_fitness, best_fitness_history